# Affine registration

This notebook presents how series of source images can be fixed by detecting affine transformations between them and applying the shift.

It is usefull in terms of model fitting to skip the affine transformation part and focus on pure fitting.


## Raw data fetch

In [2]:
import AFQ.data as afd
data_dict = afd.fetch_hcp([994273])